In [52]:
'''
Jiayou Qin 03/17/2024
SIFT based stablizer
a simple demo of stablization using sift
a naive approach is used(mean average value)
'''


import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
 
video_path = "stationary.mp4" #change this file name as needed
cap = cv.VideoCapture(video_path)

# cv.namedWindow('image', cv.WINDOW_NORMAL)
for i in range(1):
    success, frame = cap.read()
sift = cv.SIFT_create()
frame = cv.resize(frame,(800,600))
center = np.array((400,300),dtype=float)
cursor = center.copy()
last_frame = frame
kp, des = sift.detectAndCompute(frame,None)
cv.imshow('image', frame)
cv.waitKey(0) #press key to begin
while True:
    for i in range(5):
        success, frame = cap.read()
    if not success:
        break
    delta = np.array([],dtype=float)
    frame = cv.resize(frame,(800,600))
    out_frame = frame.copy()
    if not success:
        break
    kp_,des_ = kp,des
    kp, des = sift.detectAndCompute(frame,None)

    bf = cv.BFMatcher()
    matches = bf.knnMatch(des,des_,k=2)

    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            p1 = np.array(kp[m.queryIdx].pt, dtype=float)
            p2 = np.array(kp_[m.trainIdx].pt, dtype=float)
            delta = np.concatenate((delta,p2-p1))
#             cv.arrowedLine(out_frame,p1,p2,(0,0,255),1)
    delta = delta.reshape(-1,2)
    mean = np.mean(delta, axis=0)
    vec = center + mean
    
    cursor -= mean
    cv.arrowedLine(out_frame,center.astype(int),cursor.astype(int),(0,0,255),3)
    cv.circle(out_frame, cursor.astype(int), 30, (0,0,255), 3)

    cv.imshow('image', out_frame)
    k = cv.waitKey(5)
    if k ==27:
        break
    
    last_frame = frame
    
#     break;
cap.release()
cv.destroyAllWindows()
